In [1]:
import pandas as pd
import os
import json
import numpy as np
from scipy.stats import entropy
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import time

In [2]:
def check_accuracy(df, label):
    try:
        features = [col for col in df.columns if col != label]
        X = df[features]
        y = df[label]
        train_x, test_x, train_y, test_y = train_test_split(X, y, random_state=1)
        tree = HistGradientBoostingClassifier(random_state = 0)
        tree.fit(train_x, train_y)
        y_pred_tree = tree.predict(test_x)
        score_tree = accuracy_score(test_y,y_pred_tree)
        return round(score_tree*100,2)
    except Exception as e:
        return e
    
def get_entropy(df):
    i = 0
    total = 0
    for column in df.columns:
        counts = df[column].value_counts()
        prob = counts/len(df)
        entropy_value = entropy(prob, base = 2)
        total += entropy_value
        i += 1
    shannon_entropy = total/i
    return shannon_entropy

def check_changes(df_start, df_end):
    # Different shapes => cell removed/added
    if df_start.shape != df_end.shape:
        row_diff = df_end.shape[0] - df_start.shape[0]
        cols_diff = df_end.shape[1] - df_start.shape[1]
        differences = row_diff * df_start.shape[1] + cols_diff * df_start.shape[0]
    # Same shape => cell modified
    else:
        differences = (df_start != df_end).sum().sum()
    return differences

## Pipeline1

In [4]:
df = pd.read_csv(r'C:\Users\rocci\OneDrive\Desktop\Università\TesiMagistrale\dataPreparationTool\testing\census.csv')
accuracy_start = check_accuracy(df, '14')
completeness_start = df.count()/df.shape[0]
shannon_start = get_entropy(df)
df_start = df
start_time = time.time_ns()

df.columns = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'label']
columns = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country', 'label']
df_modified = df.replace('?', np.nan)
df_modified[columns] = df_modified[columns].applymap(str.strip)
columns = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'native-country']
for i, col in enumerate(columns):   
    dummies = pd.get_dummies(df_modified[col])
    df_dummies = dummies.add_prefix(col + '_')
    df_modified = df_modified.join(df_dummies)        
    df_modified = df_modified.drop([col], axis=1)
df_modified = df_modified.replace({'sex': {'Male': 1, 'Female': 0}, 'label': {'<=50K': 0, '>50K': 1}})
df_modified = df_modified.drop(['fnlwgt'], axis=1)

end_time = time.time_ns()
total_time = end_time - start_time
completeness_end = df_modified.count()/df_modified.shape[0]

accuracy_end = check_accuracy(df_modified, 'label')

shannon_end = get_entropy(df_modified)
differences = check_changes(df_start, df_modified)
response = (
    'Pipeline Execution Time: ' + str(total_time / 1_000_000_000) + 's,\n' +
    'Completeness Before Pipeline: ' + str(completeness_start.mean(numeric_only=True) * 100) + '%,\n' +
    'Completeness After Pipeline: ' + str(completeness_end.mean(numeric_only=True) * 100) + '%,\n' +
    "Average Shannon's Entropy Before Pipeline: " + str(shannon_start) + ',\n' +
    "Average Shannon's Entropy After Pipeline: " + str(shannon_end) + ',\n' +
    'Accuracy on a ML Algorithm Before Pipeline: ' + str(accuracy_start) + ',\n' +
    'Accuracy on a ML Algorithm After Pipeline: ' + str(accuracy_end) + ',\n' +
    '# Cells Modified: ' + str(differences)
)

df_modified.to_csv(r'C:\Users\rocci\OneDrive\Desktop\Università\TesiMagistrale\dataPreparationTool\data\output\output1.csv', index = False)
print(response)

Pipeline Execution Time: 0.5340027s,
Completeness Before Pipeline: 100.0%,
Completeness After Pipeline: 100.0%,
Average Shannon's Entropy Before Pipeline: 2.8781066070808152,
Average Shannon's Entropy After Pipeline: 0.3373531744120834,
Accuracy on a ML Algorithm Before Pipeline: could not convert string to float: ' Private',
Accuracy on a ML Algorithm After Pipeline: 87.57,
# Cells Modified: 2995612


## Pipeline2

In [17]:
df = pd.read_csv(r'C:\Users\rocci\OneDrive\Desktop\Università\TesiMagistrale\dataPreparationTool\testing\compas.csv')
accuracy_start = check_accuracy(df, 'two_year_recid')
completeness_start = df.count()/df.shape[0]
shannon_start = get_entropy(df)
df_start = df
start_time = time.time_ns()

columns = ['age', 'c_charge_degree', 'race', 'sex', 'priors_count', 'days_b_screening_arrest', 'two_year_recid', 'c_jail_in', 'c_jail_out']
df_modified = df.drop(df.columns.difference(columns), axis=1)
df_modified = df_modified.dropna()
df_modified['race'] = [0 if r != 'Caucasian' else 1 for r in df_modified['race']]
df_modified = df_modified.rename({'two_year_recid': 'label'}, axis=1)
df_modified['label'] = [0 if l == 1 else 1 for l in df_modified['label']]
df_modified['sex'] = df_modified['sex'].replace({'Male': 1, 'Female': 0})
df_modified['jailtime'] = (pd.to_datetime(df_modified.c_jail_out) - pd.to_datetime(df_modified.c_jail_in)).dt.days
df_modified = df_modified.drop(['c_jail_in', 'c_jail_out'], axis=1)
df_modified['c_charge_degree'] = [0 if s == 'M' else 1 for s in df_modified['c_charge_degree']]

end_time = time.time_ns()
total_time = end_time - start_time
completeness_end = df_modified.count()/df_modified.shape[0]

accuracy_end = check_accuracy(df_modified, 'label')

shannon_end = get_entropy(df_modified)
differences = check_changes(df_start, df_modified)
response = (
    'Pipeline Execution Time: ' + str(total_time / 1_000_000_000) + 's,\n' +
    'Completeness Before Pipeline: ' + str(completeness_start.mean(numeric_only=True) * 100) + '%,\n' +
    'Completeness After Pipeline: ' + str(completeness_end.mean(numeric_only=True) * 100) + '%,\n' +
    "Average Shannon's Entropy Before Pipeline: " + str(shannon_start) + ',\n' +
    "Average Shannon's Entropy After Pipeline: " + str(shannon_end) + ',\n' +
    'Accuracy on a ML Algorithm Before Pipeline: ' + str(accuracy_start) + ',\n' +
    'Accuracy on a ML Algorithm After Pipeline: ' + str(accuracy_end) + ',\n' +
    '# Cells Modified: ' + str(differences)
)

df_modified.to_csv(r'C:\Users\rocci\OneDrive\Desktop\Università\TesiMagistrale\dataPreparationTool\data\output\output2.csv', index = False)
print(response)

Pipeline Execution Time: 0.0411182s,
Completeness Before Pipeline: 81.37269774181232%,
Completeness After Pipeline: 100.0%,
Average Shannon's Entropy Before Pipeline: 5.599599641948954,
Average Shannon's Entropy After Pipeline: 2.4483103142809326,
Accuracy on a ML Algorithm Before Pipeline: could not convert string to float: 'stephanie nevels',
Accuracy on a ML Algorithm After Pipeline: 66.82,
# Cells Modified: -340901


## PIPELINE3

In [10]:
df = pd.read_csv(r'C:\Users\rocci\OneDrive\Desktop\Università\TesiMagistrale\dataPreparationTool\testing\german.csv')
accuracy_start = check_accuracy(df, 'label')
completeness_start = df.count()/df.shape[0]
shannon_start = get_entropy(df)
df_start = df
start_time = time.time_ns()

df = df.replace({'checking': {'A11': 'check_low', 'A12': 'check_mid', 'A13': 'check_high',
                                  'A14': 'check_none'},
                     'credit_history': {'A30': 'debt_none', 'A31': 'debt_noneBank',
                                        'A32': 'debt_onSchedule', 'A33': 'debt_delay',
                                        'A34': 'debt_critical'},
                     'purpose': {'A40': 'pur_newCar', 'A41': 'pur_usedCar',
                                 'A42': 'pur_furniture', 'A43': 'pur_tv',
                                 'A44': 'pur_appliance', 'A45': 'pur_repairs',
                                 'A46': 'pur_education', 'A47': 'pur_vacation',
                                 'A48': 'pur_retraining', 'A49': 'pur_business',
                                 'A410': 'pur_other'},
                     'savings': {'A61': 'sav_small', 'A62': 'sav_medium', 'A63': 'sav_large',
                                 'A64': 'sav_xlarge', 'A65': 'sav_none'},
                     'employment': {'A71': 'emp_unemployed', 'A72': 'emp_lessOne',
                                    'A73': 'emp_lessFour', 'A74': 'emp_lessSeven',
                                    'A75': 'emp_moreSeven'},
                     'other_debtors': {'A101': 'debtor_none', 'A102': 'debtor_coApp',
                                       'A103': 'debtor_guarantor'},
                     'property': {'A121': 'prop_realEstate', 'A122': 'prop_agreement',
                                  'A123': 'prop_car', 'A124': 'prop_none'},
                     'other_inst': {'A141': 'oi_bank', 'A142': 'oi_stores', 'A143': 'oi_none'},
                     'housing': {'A151': 'hous_rent', 'A152': 'hous_own', 'A153': 'hous_free'},
                     'job': {'A171': 'job_unskilledNR', 'A172': 'job_unskilledR',
                             'A173': 'job_skilled', 'A174': 'job_highSkill'},
                     'phone': {'A191': 0, 'A192': 1},
                     'foreigner': {'A201': 1, 'A202': 0},
                     'label': {2: 0}})
df['status'] = np.where(df.personal_status == 'A91', 'divorced',
                            np.where(df.personal_status == 'A92', 'divorced',
                                     np.where(df.personal_status == 'A93', 'single',
                                              np.where(df.personal_status == 'A95', 'single',
                                                       'married'))))
df['gender'] = np.where(df.personal_status == 'A92', 0, np.where(df.personal_status == 'A95', 0, 1))
df_modified = df.drop(['personal_status'], axis=1)
columns = ['checking', 'credit_history', 'purpose', 'savings', 'employment', 'other_debtors', 'property',
           'other_inst', 'housing', 'job', 'status']
for i, col in enumerate(columns):
    dummies = pd.get_dummies(df_modified[col])
    df_dummies = dummies.add_prefix(col + '_')
    df_modified = df_modified.join(df_dummies)
    df_modified = df_modified.drop([col], axis=1)

end_time = time.time_ns()
total_time = end_time - start_time
completeness_end = df_modified.count()/df_modified.shape[0]

accuracy_end = check_accuracy(df_modified, 'label')

shannon_end = get_entropy(df_modified)
differences = check_changes(df_start, df_modified)
response = (
    'Pipeline Execution Time: ' + str(total_time / 1000000000) + 's,\n' +
    'Completeness Before Pipeline: ' + str(completeness_start.mean(numeric_only=True) * 100) + '%,\n' +
    'Completeness After Pipeline: ' + str(completeness_end.mean(numeric_only=True) * 100) + '%,\n' +
    "Average Shannon's Entropy Before Pipeline: " + str(shannon_start) + ',\n' +
    "Average Shannon's Entropy After Pipeline: " + str(shannon_end) + ',\n' +
    'Accuracy on a ML Algorithm Before Pipeline: ' + str(accuracy_start) + ',\n' +
    'Accuracy on a ML Algorithm After Pipeline: ' + str(accuracy_end) + ',\n' +
    '# Cells Modified: ' + str(differences)
)

df_modified.to_csv(r'C:\Users\rocci\OneDrive\Desktop\Università\TesiMagistrale\dataPreparationTool\data\output\output3.csv', index = False)
print(response)

Pipeline Execution Time: 0.1070304s,
Completeness Before Pipeline: 100.0%,
Completeness After Pipeline: 100.0%,
Average Shannon's Entropy Before Pipeline: 2.084195887922238,
Average Shannon's Entropy After Pipeline: 0.9285827754815593,
Accuracy on a ML Algorithm Before Pipeline: could not convert string to float: 'A14',
Accuracy on a ML Algorithm After Pipeline: 74.4,
# Cells Modified: 39000


## Pipeline4

In [29]:
df = pd.read_csv(r'C:\Users\rocci\OneDrive\Desktop\Università\TesiMagistrale\dataPreparationTool\testing\transfusion.csv')
accuracy_start = check_accuracy(df, 'Class')
completeness_start = df.count()/df.shape[0]
shannon_start = get_entropy(df)
start_time = time.time_ns()

df['Class'] = df['Class'].replace({1: 0, 2: 1})
label = df['Class']
df = df.drop(['Class'], axis = 1)
columns = ['V1', 'V2', 'V3', 'V4']
for i, col in enumerate(columns):
    dummies = pd.get_dummies(df[col])
    df_dummies = dummies.add_prefix(col + '_')
    df = df.join(df_dummies)
    df = df.drop([col], axis=1)
imputer = SimpleImputer(strategy='median', copy=True)
df = imputer.fit_transform(df)
df_modified = pd.DataFrame(df)
scaler = StandardScaler(with_mean=False, with_std=True)
columns = df_modified.columns[1:]
df_modified[columns] = scaler.fit_transform(df_modified[columns])
df_modified['Class'] = label

#df_modified = df_modified.astype(df_start.dtypes)


end_time = time.time_ns()
total_time = end_time - start_time
completeness_end = df_modified.count()/df_modified.shape[0]
df_start = pd.read_csv(r'C:\Users\rocci\OneDrive\Desktop\Università\TesiMagistrale\dataPreparationTool\testing\transfusion.csv')
accuracy_end = check_accuracy(df_modified, 'Class')

shannon_end = get_entropy(df_modified)
differences = check_changes(df_start, df_modified)
response = (
    'Pipeline Execution Time: ' + str(total_time / 1000000000) + 's,\n' +
    'Completeness Before Pipeline: ' + str(completeness_start.mean(numeric_only=True) * 100) + '%,\n' +
    'Completeness After Pipeline: ' + str(completeness_end.mean(numeric_only=True) * 100) + '%,\n' +
    "Average Shannon's Entropy Before Pipeline: " + str(shannon_start) + ',\n' +
    "Average Shannon's Entropy After Pipeline: " + str(shannon_end) + ',\n' +
    'Accuracy on a ML Algorithm Before Pipeline: ' + str(accuracy_start) + ',\n' +
    'Accuracy on a ML Algorithm After Pipeline: ' + str(accuracy_end) + ',\n' +
    '# Cells Modified: ' + str(differences)
)

df_modified.to_csv(r'C:\Users\rocci\OneDrive\Desktop\Università\TesiMagistrale\dataPreparationTool\data\output\output4.csv', index = False)
print(response)
print(df_modified.shape)
print(df_start.shape)

C:\Users\rocci\AppData\Local\Temp\ipykernel_38140\1132282002.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_modified['Class'] = label


Pipeline Execution Time: 0.1287323s,
Completeness Before Pipeline: 100.0%,
Completeness After Pipeline: 100.0%,
Average Shannon's Entropy Before Pipeline: 4.428688739721966,
Average Shannon's Entropy After Pipeline: 0.18106952412414623,
Accuracy on a ML Algorithm Before Pipeline: 74.33,
Accuracy on a ML Algorithm After Pipeline: 77.54,
# Cells Modified: 127908
(748, 177)
(748, 6)


## Pipeline5

In [24]:
df = pd.read_csv(r'C:\Users\rocci\OneDrive\Desktop\Università\TesiMagistrale\dataPreparationTool\testing\customers.csv')
accuracy_start = check_accuracy(df, 'Channel')
completeness_start = df.count()/df.shape[0]
shannon_start = get_entropy(df)
df_modified = pd.DataFrame()
start_time = time.time_ns()

scaler = StandardScaler(with_mean=False, with_std=True)
columns = ['V2', 'V3', 'V4', 'V5', 'V6', 'V7']
df[columns] = scaler.fit_transform(df[columns])
df['Channel'] = df['Channel'].replace({1: 0, 2: 1})

end_time = time.time_ns()
total_time = end_time - start_time
df_modified = df
completeness_end = df_modified.count()/df_modified.shape[0]
df_start = pd.read_csv(r'C:\Users\rocci\OneDrive\Desktop\Università\TesiMagistrale\dataPreparationTool\testing\customers.csv')
accuracy_end = check_accuracy(df_modified, 'Channel')

shannon_end = get_entropy(df_modified)
differences = check_changes(df_start, df_modified)
response = (
    'Pipeline Execution Time: ' + str(total_time / 1_000_000_000) + 's,\n' +
    'Completeness Before Pipeline: ' + str(completeness_start.mean(numeric_only=True) * 100) + '%,\n' +
    'Completeness After Pipeline: ' + str(completeness_end.mean(numeric_only=True) * 100) + '%,\n' +
    "Average Shannon's Entropy Before Pipeline: " + str(shannon_start) + ',\n' +
    "Average Shannon's Entropy After Pipeline: " + str(shannon_end) + ',\n' +
    'Accuracy on a ML Algorithm Before Pipeline: ' + str(accuracy_start) + ',\n' +
    'Accuracy on a ML Algorithm After Pipeline: ' + str(accuracy_end) + ',\n' +
    '# Cells Modified: ' + str(differences)
)

df_modified.to_csv(r'C:\Users\rocci\OneDrive\Desktop\Università\TesiMagistrale\dataPreparationTool\data\output\output5.csv', index = False)
print(response)
print(df_start.shape)
print(df_modified.shape)

Pipeline Execution Time: 0.0099998s,
Completeness Before Pipeline: 100.0%,
Completeness After Pipeline: 100.0%,
Average Shannon's Entropy Before Pipeline: 6.411632038013595,
Average Shannon's Entropy After Pipeline: 6.411632038013595,
Accuracy on a ML Algorithm Before Pipeline: 92.73,
Accuracy on a ML Algorithm After Pipeline: 92.73,
# Cells Modified: 3080
(440, 10)
(440, 10)


## Pipeline6

In [4]:
df = pd.read_csv(r'C:\Users\rocci\OneDrive\Desktop\Università\TesiMagistrale\dataPreparationTool\testing\cleveland.csv')
df.nunique()

id            303
'age'          41
'sex'           2
'cp'            4
'trestbps'     50
'chol'        152
'fbs'           2
'restecg'       3
'thalach'      91
'exang'         2
'oldpeak'      40
'slope'         3
'ca'            5
'thal'          4
'num'           5
dtype: int64

In [33]:
df = pd.read_csv(r'C:\Users\rocci\OneDrive\Desktop\Università\TesiMagistrale\dataPreparationTool\testing\cleveland.csv')
df_start = pd.read_csv(r'C:\Users\rocci\OneDrive\Desktop\Università\TesiMagistrale\dataPreparationTool\testing\cleveland.csv')
df = df.replace({'?': np.nan})
accuracy_start = check_accuracy(df, "'fbs'")
completeness_start = df.count()/df.shape[0]
shannon_start = get_entropy(df)
start_time = time.time_ns()

imputer = SimpleImputer(strategy='most_frequent', copy=True)
df = imputer.fit_transform(df)
df_modified = pd.DataFrame(df)
scaler = StandardScaler(with_mean=False, with_std=True)
columns = df_modified.columns[1:-1]
df_modified[columns] = scaler.fit_transform(df_modified[columns])
df_modified.columns = df_start.columns
df_modified = df_modified.astype(df_start.dtypes)
columns = ["'ca'", "'slope'"]
for i, col in enumerate(columns):
        dummies = pd.get_dummies(df_modified[col])
        df_dummies = dummies.add_prefix(col + '_')
        df_modified = df_modified.join(df_dummies)
        df_modified = df_modified.drop([col], axis=1)


end_time = time.time_ns()
total_time = end_time - start_time
completeness_end = df_modified.count()/df_modified.shape[0]
accuracy_end = check_accuracy(df_modified, "'fbs'")


shannon_end = get_entropy(df_modified)
differences = check_changes(df_start, df_modified)
response = (
    'Pipeline Execution Time: ' + str(total_time / 1_000_000_000) + 's,\n' +
    'Completeness Before Pipeline: ' + str(completeness_start.mean(numeric_only=True) * 100) + '%,\n' +
    'Completeness After Pipeline: ' + str(completeness_end.mean(numeric_only=True) * 100) + '%,\n' +
    "Average Shannon's Entropy Before Pipeline: " + str(shannon_start) + ',\n' +
    "Average Shannon's Entropy After Pipeline: " + str(shannon_end) + ',\n' +
    'Accuracy on a ML Algorithm Before Pipeline: ' + str(accuracy_start) + ',\n' +
    'Accuracy on a ML Algorithm After Pipeline: ' + str(accuracy_end) + ',\n' +
    '# Cells Modified: ' + str(differences)
)

df_modified.to_csv(r'C:\Users\rocci\OneDrive\Desktop\Università\TesiMagistrale\dataPreparationTool\data\output\output6.csv', index = False)
print(response)
print(df_start.shape)
print(df_modified.shape)

C:\Users\rocci\AppData\Local\Temp\ipykernel_27888\223515889.py:19: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  dummies = pd.get_dummies(df_modified[col])


Pipeline Execution Time: 0.0430057s,
Completeness Before Pipeline: 99.86798679867987%,
Completeness After Pipeline: 100.0%,
Average Shannon's Entropy Before Pipeline: 3.104902911182914,
Average Shannon's Entropy After Pipeline: 1.6911882470387252,
Accuracy on a ML Algorithm Before Pipeline: 78.95,
Accuracy on a ML Algorithm After Pipeline: 82.89,
# Cells Modified: 1515
(303, 15)
(303, 20)
